# Задание по программированию: Размещение баннеров

Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

Скачаем любым удобным образом архив fsq.zip с этой страницы.

Нас будет интересовать файл checkins.dat

In [149]:
# импортируем все библиотеки здесь
import pandas as pd
from sklearn.cluster import MeanShift
import numpy as np
from collections import Counter
from scipy.spatial import distance

Для удобной работы с этим документом преобразуем его к формату csv, удалив строки, не содержащие координат — они неинформативны для нас:

In [77]:
# Преобразуем к формату csv и посмотрим, как выглядят данные
df = pd.read_csv('checkins.dat', sep='|', header=0, skipinitialspace=True)
df.head()

C:\Users\Svetlana\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,user_id,venue_id,latitude,longitude,created_at
0,---------+---------+----------+---------------...,NaN,NaN,NaN,NaN,NaN
1,984301,2041916.0,5222.0,NaN,NaN,2012-04-21 17:39:01
2,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
3,984315,1764391.0,5222.0,NaN,NaN,2012-04-21 17:37:18
4,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43


In [78]:
# Посмотрим на названия столбцов
df.columns

Index(['id    ', 'user_id ', 'venue_id ', 'latitude      ', 'longitude     ',
       'created_at      '],
      dtype='object')

In [79]:
# Почистим названия, уберем пробелы
old_columns = df.columns.tolist()
df.set_axis([''.join(s.split()) for s in old_columns], axis = 'columns', inplace = True)
df.columns

Index(['id', 'user_id', 'venue_id', 'latitude', 'longitude', 'created_at'], dtype='object')

In [80]:
# Удалим все строки, где отсутствуют значения координат
df.dropna(subset = ['latitude','longitude'], inplace = True)
df.reset_index(inplace = True, drop = True)
df.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
1,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
2,984291,105054.0,5222.0,45.523452,-122.676207,2012-04-21 17:39:22
3,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46
4,984232,93870.0,380645.0,33.448377,-112.074037,2012-04-21 17:38:18


С помощью pandas построим DataFrame и убедимся, что все 396634 строки с координатами считаны успешно.

In [81]:
df.shape

(396634, 6)

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396634 entries, 0 to 396633
Data columns (total 6 columns):
id            396634 non-null object
user_id       396634 non-null float64
venue_id      396634 non-null float64
latitude      396634 non-null float64
longitude     396634 non-null float64
created_at    396634 non-null object
dtypes: float64(4), object(2)
memory usage: 18.2+ MB


Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в [sklearn user guide](https://scikit-learn.org/stable/modules/clustering.html#mean-shift), а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

__Примечание:__ на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.

In [83]:
# Отрежем первые 100 тысяч строк
mdf = df.head(100000)
mdf.shape

(100000, 6)

In [84]:
mdf.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
1,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
2,984291,105054.0,5222.0,45.523452,-122.676207,2012-04-21 17:39:22
3,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46
4,984232,93870.0,380645.0,33.448377,-112.074037,2012-04-21 17:38:18


In [87]:
# Сформируем массив для кластеризации
lat_array = np.array(mdf.latitude.to_list())
long_array = np.array(mdf.longitude.to_list())
coordinates = np.column_stack((lat_array, long_array))
coordinates

array([[  38.8951118,  -77.0363658],
       [  33.800745 ,  -84.41052  ],
       [  45.5234515, -122.6762071],
       ...,
       [  29.7628844,  -95.3830615],
       [  32.802955 ,  -96.769923 ],
       [  37.7749295, -122.4194155]])

In [88]:
clustering = MeanShift(bandwidth=0.1).fit(coordinates)

In [108]:
labels = clustering.labels_
centers = clustering.cluster_centers_

print("Количество кластеров : %d" % len(centers))

Количество кластеров : 3231


In [99]:
len(clustering.cluster_centers_)

3231

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.\
При желании увидеть получившиеся результаты на карте можно передать центры получившихся кластеров в один из инструментов визуализации. Например, сайт mapcustomizer.com имеет функцию Bulk Entry, куда можно вставить центры полученных кластеров.

In [109]:
# Посчитаем количество точек в каждом кластере
count_clusters = Counter()
for label in labels:
    count_clusters[label] += 1


# Получим список кластеров, где точек больше 15

big_clusters = []

for cluster in count_clusters:
    if count_clusters[cluster] > 15:
        big_clusters.append(cluster)

# Посмотрим, сколько таких кластеров получилось
len(big_clusters)

592

Как мы помним, 20 баннеров надо разместить близ офисов компании.

In [140]:
# Сохраним все координаты в файл Carnival_Cruise_Line_coordinates.txt. Считаем его, и преобразуем в матрицу, а потом в np.array
CCL_coordinates = []

with open('Carnival_Cruise_Line_coordinates.txt', encoding='utf-8') as f:
    
    # Получим список строк в файле, кроме пустых
    for i in [line for line in f.readlines() if line > '\n']:
        
        # Преобразуем их в список
        spam = i.split()
        
        # Очистим от лишних запятых
        for ind, val in enumerate(spam):
            spam[ind] = val.strip(',')
            
        # Добавим только координаты в матрицу координат
        CCL_coordinates.extend([spam[:2]])

# Посмотрим на результат
CCL_coordinates

[['33.751277', '-118.188740'],
 ['25.867736', '-80.324116'],
 ['51.503016', '-0.075479'],
 ['52.378894', '4.885084'],
 ['39.366487', '117.036146'],
 ['-33.868457', '151.205134']]

In [148]:
# Преобразуем матрицу в np.array
np.set_printoptions(suppress=True)
CCL_coordinates = np.array(CCL_coordinates, dtype = 'float64')

CCL_coordinates

array([[  33.751277, -118.18874 ],
       [  25.867736,  -80.324116],
       [  51.503016,   -0.075479],
       [  52.378894,    4.885084],
       [  39.366487,  117.036146],
       [ -33.868457,  151.205134]])

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

In [152]:
# Преобразуем массив с координатами центров кластеров в np.array
centers = np.array(centers)

centers

array([[  40.7177164 ,  -73.99183542],
       [  33.44943805, -112.00213969],
       [  33.44638027, -111.90188756],
       ...,
       [ -37.8229826 ,  145.1811902 ],
       [ -41.2924945 ,  174.7732353 ],
       [ -45.0311622 ,  168.6626435 ]])

In [156]:
# Найдем расстояние между центрами кластеров и офисами с помощью scipy
np.set_printoptions(linewidth = 150)

distance_between_CCL_and_clusters = distance.cdist(centers, CCL_coordinates, 'euclidean')

distance_between_CCL_and_clusters

array([[ 44.74257092,  16.14371999,  74.69906582,  79.73425538, 191.0327603 , 237.22725876],
       [  6.19395917,  32.5726786 , 113.37331719, 118.41008135, 229.11470442, 271.67953165],
       [  6.29424146,  32.47447468, 113.2748328 , 118.31160891, 229.01456477, 271.58164907],
       ...,
       [272.92232369, 234.32701671, 170.52458466, 166.79141452,  82.16056067,   7.20598166],
       [302.42071128, 263.78998313, 197.94716381, 194.00080664,  99.19396543,  24.70974973],
       [297.47334157, 258.88426019, 194.40010686, 190.55657439,  98.93562773,  20.72126023]])

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.

In [170]:
# Найдем минимальное расстояние между офисами и центрами кластеров
dist_min = np.amin(distance_between_CCL_and_clusters, axis = None)

# Получим индекс этого минимального расстояния
i, j = np.where(distance_between_CCL_and_clusters == dist_min)

# Номер строки в матрице distance_between_CCL_and_clusters и 
# номер строки в матрице координат центров кластеров centers совпадают.
i[0]

420

In [174]:
# Запишем ответ и сдадим работу
with open('C3_W1.txt', 'w') as f:
    f.write(' '.join(str(num) for num in centers[i[0]]))

Теперь найдем координаты 20 ближайших кластеров. 
К сожалению, готовой реализации я не нашла, сделаем свой вариант.
Мы сожмем distance_between_CCL_and_clusters до одной оси, отсортируем, выберем 20 минимальных значений 
и найдем координаты центров кластеров, как мы сделали выше.

In [187]:
min_distances = np.sort(distance_between_CCL_and_clusters.reshape(-1))[:20]
min_distances

array([0.00783476, 0.00935332, 0.02267407, 0.05005829, 0.07084773, 0.13410903, 0.15410283, 0.16740596, 0.18887596, 0.19222727, 0.19577946,
       0.21181054, 0.22223329, 0.24065315, 0.25150209, 0.26892863, 0.27130076, 0.27531986, 0.28971896, 0.29497889])

In [189]:
nearest_clusters = []
for dist in min_distances:
    i, j = np.where(distance_between_CCL_and_clusters == dist)
    nearest_clusters.append(centers[i[0]])

nearest_clusters    

[array([-33.86063043, 151.20477593]),
 array([52.37296399,  4.89231722]),
 array([ 25.84567226, -80.3188906 ]),
 array([51.50299126, -0.12553729]),
 array([  33.80987796, -118.14892381]),
 array([ 25.78581242, -80.21793804]),
 array([-34.00190615, 151.12806905]),
 array([ 25.70534972, -80.28342874]),
 array([ 26.01009825, -80.19999059]),
 array([-33.9522629, 151.0321372]),
 array([  33.88832534, -118.04892817]),
 array([  33.87298601, -118.36209115]),
 array([  33.97257482, -118.16837067]),
 array([51.42676329, -0.30373207]),
 array([52.388501  ,  4.63376547]),
 array([51.5741517,  0.1838708]),
 array([ 26.13884379, -80.33434684]),
 array([52.2644,  4.6347]),
 array([51.50647877, -0.36517727]),
 array([  33.98393587, -118.00740497])]

In [190]:
# Первый центр из nearest_clusters должен иметь индекс 420 (как в ответе на задание).
np.where(centers == nearest_clusters[0])

(array([420, 420], dtype=int64), array([0, 1], dtype=int64))